In [57]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained('gpt2')
model = AutoModelForCausalLM.from_pretrained('gpt2')

In [ ]:
# prompt = 'the dog walked the man to the park'

# tokens = tokenizer.encode(prompt, return_tensors='pt')
# print(tokens)
# print(tokenizer.tokenize(prompt))
# outputs = model(tokens, output_attentions=True)
# print(len(outputs.attentions))
# outputs.attentions[0].shape

In [ ]:
# from bertviz import model_view, head_view

# model_view(outputs.attentions, tokenizer.convert_ids_to_tokens(tokens[0]))
# head_view(outputs.attentions, tokenizer.convert_ids_to_tokens(tokens[0]))

In [ ]:
# Goal:
# 1. Take a prompt.
# 2. Generate an answer to the question.
# 3. Quantify where attention was paid on each token of the answer.
# 4. Visualize that attention. Visualization should work for long passages.

In [107]:
def inspect_shape(name, obj):
    # Since this is the top-level inspect call, print the name of the variable.
    print(f'{name}: ', end='')
    _inspect_shape(obj, len(name) + 2)  # + 2 for ': '


def _class_name(obj):
    return type(obj).__name__


def _inspect_shape(obj, indent):
    """ 
    Print the given obj. Does not print preceding name or preceding indentation.
    Param `indent` should be the indentation level where this obj is being printed. Will be used to print nested properties if necessary.
    """

    # Base Cases (sequences with known-typed contents, tensors, other objects or primitives)
    if isinstance(obj, str):
        print(f'{_class_name(obj)}[len={len(obj)}]')
    elif isinstance(obj, torch.Tensor):
        print(f'{_class_name(obj)}[shape={list(obj.shape)}, sum={obj.sum()}]')
    else:

        # Dict where contents should be recursively inspected
        try:
            print(f'{_class_name(obj)}[len={len(obj.items())}]')
            for key, val in obj.items():
                print(' ' * (indent + 2) + f'{key}: ', end='')
                _inspect_shape(val, indent + len(key) + 2)  # + 2 for ': '
        except (TypeError, AttributeError):
            # Sequence where contents should be recursively inspected
            try:
                print(f'{_class_name(obj)}[len={len(obj)}]')
                for i in range(min(len(obj), 3)):
                    print(' ' * (indent + 2) + f'{i}: ', end='')
                    # + 2 for ': '
                    _inspect_shape(obj[i], indent + len(str(i)) + 2)
            except (TypeError, AttributeError):
                # Default case if nothing else works
                print(f'{_class_name(obj)}')

In [111]:
import altair as alt
import pandas as pd


def attn_viz(prompt_tokens, generated_sequence_tokens, generated_token_avg_attns):
    """
    prompt_tokens should be simply the token IDs from the prompt
    generated_sequence should be the raw sequence from huggingface .generate(), including both prompt and generated tokens.
    generated_token_avg_attns should be a sequence with len(generated tokens), where each element is a sequence with len(tokens through generated token, including prompt)
    """
    df = _prepare_attn_df(prompt_tokens, generated_sequence_tokens,
                          generated_token_avg_attns)
    _show_attn_chart(df)


def _prepare_attn_df(prompt_tokens, generated_sequence_tokens, generated_token_avg_attns):
    df = pd.DataFrame({'token_id': generated_sequence_tokens, 'token_text': map(
        tokenizer.decode, generated_sequence_tokens)})
    for i in range(len(generated_sequence_tokens)):
        df[f'token_attn_from_{i}'] = 0.0
    display(df)


def _show_attn_chart(df):
    """
    df should have columns:
      token_id: int representation of token
      token_text: string representation of token
      token_attn_from_{i}: float representing the attention placed on this token by the token with index i in the sequence
    """
    pass

In [112]:
def average_token_attn(token_attn):
    layer_avg_attns = []
    for layer_idx, layer_attn in enumerate(token_attn):
        print('layer_idx', layer_idx)
        # Take mean across model's attention heads
        layer_avg_attn = layer_attn.squeeze(0).mean(dim=0)
        # inspect_shape('layer_avg_attn', layer_avg_attn)
        layer_avg_attn_cleaned = torch.concat([
            torch.tensor([0.]),  # First entry is null attention, set it to 0
            # Remove all tokens except the most recent (since the first token generated has entire prompts' worth of attention generated) and remove the first entry
            layer_avg_attn[-1][1:],
            torch.tensor([0.]),  # Add a 0 for the current token itself
        ])
        # inspect_shape('layer_avg_attn_cleaned', layer_avg_attn_cleaned)
        layer_avg_attn_normalized = layer_avg_attn_cleaned / layer_avg_attn_cleaned.sum()
        # inspect_shape('layer_avg_attn_normalized', layer_avg_attn_normalized)
        layer_avg_attns.append(layer_avg_attn_normalized)
    # inspect_shape('layer_avg_attns', layer_attn)
    return torch.stack(layer_avg_attns).mean(dim=0)


def generate_and_attn(prompt):
    prompt_tokens = tokenizer.encode(prompt, return_tensors='pt')
    inspect_shape('tokens', prompt_tokens)

    outputs = model.generate(prompt_tokens, max_new_tokens=48,
                             output_attentions=True, return_dict_in_generate=True, pad_token_id=tokenizer.eos_token_id)
    inspect_shape('outputs', outputs)
    # token_avg_attn = average_token_attn(outputs.attentions[1])
    # inspect_shape('token_avg_attn', token_avg_attn)
    # print(token_avg_attn)
    # for token in outputs.sequences[0]:
    #     print(token, tokenizer.decode(token), end=', ')
    attn_viz(prompt_tokens, outputs.sequences[0], map(
        average_token_attn, outputs.attentions))


PROMPT = """
The 2008 Summer Olympics torch relay was run from March 24 until August 8, 2008, prior to the 2008 Summer Olympics, with the theme of "one world, one dream". The torch relay took place over 45km with 18 total runners.

Q: What was the theme?
A:
""".strip()

generate_and_attn(PROMPT)

tokens: Tensor[shape=[1, 60], sum=254623]
outputs: GreedySearchDecoderOnlyOutput[len=3]
           sequences: Tensor[shape=[1, 108], sum=454166]
           attentions: tuple[len=48]
                       0: tuple[len=12]
                          0: Tensor[shape=[1, 12, 60, 60], sum=720.0]
                          1: Tensor[shape=[1, 12, 60, 60], sum=720.0]
                          2: Tensor[shape=[1, 12, 60, 60], sum=720.0]
                       1: tuple[len=12]
                          0: Tensor[shape=[1, 12, 1, 61], sum=11.999999046325684]
                          1: Tensor[shape=[1, 12, 1, 61], sum=12.000000953674316]
                          2: Tensor[shape=[1, 12, 1, 61], sum=12.0]
                       2: tuple[len=12]
                          0: Tensor[shape=[1, 12, 1, 62], sum=12.000000953674316]
                          1: Tensor[shape=[1, 12, 1, 62], sum=12.0]
                          2: Tensor[shape=[1, 12, 1, 62], sum=11.999999046325684]
           past_key_valu

,token_id,token_text
0,464,The
1,3648,2008
2,10216,Summer
3,14935,Olympics
4,28034,torch
...,...,...
103,28034,torch
104,24248,relay
105,1718,took
106,1295,place


In [84]:
a = torch.Tensor([1, 2, 3, 4])
torch.nn.functional.normalize(a, dim=0)

tensor([0.1826, 0.3651, 0.5477, 0.7303])